In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [6]:
df = pd.read_csv('cropdata_updated.csv')

In [7]:
df.head()

,crop ID,soil_type,Seedling Stage,MOI,temp,humidity,result
0,Wheat,Black Soil,Germination,1,25,80.0,1
1,Wheat,Black Soil,Germination,2,26,77.0,1
2,Wheat,Black Soil,Germination,3,27,74.0,1
3,Wheat,Black Soil,Germination,4,28,71.0,1
4,Wheat,Black Soil,Germination,5,29,68.0,1


In [8]:
df.describe()

,MOI,temp,humidity,result
count,16411.000000,16411.000000,16411.000000,16411.000000
mean,43.695387,28.832612,63.487752,0.516178
std,27.160264,9.701465,22.630965,0.621691
min,1.000000,13.000000,15.000000,0.000000
25%,21.000000,20.000000,44.000000,0.000000
50%,41.000000,28.000000,69.200000,0.000000
75%,64.000000,37.000000,84.000000,1.000000
max,100.000000,46.000000,91.000000,2.000000


In [9]:
# encode crop ID, soil_type, Seedling Stage

le = LabelEncoder()
df['crop ID'] = le.fit_transform(df['crop ID'])
print(dict(zip(le.classes_, le.transform(le.classes_))))
df['soil_type'] = le.fit_transform(df['soil_type'])
print(dict(zip(le.classes_, le.transform(le.classes_))))
df['Seedling Stage'] = le.fit_transform(df['Seedling Stage'])
print(dict(zip(le.classes_, le.transform(le.classes_))))


{'Carrot': 0, 'Chilli': 1, 'Potato': 2, 'Tomato': 3, 'Wheat': 4}
{'Alluvial Soil': 0, 'Black Soil': 1, 'Chalky Soil': 2, 'Clay Soil': 3, 'Loam Soil': 4, 'Red Soil': 5, 'Sandy Soil': 6}
{'Flowering': 0, 'Fruit/Grain/Bulb Formation': 1, 'Germination': 2, 'Harvest': 3, 'Maturation': 4, 'Pollination': 5, 'Seedling Stage': 6, 'Vegetative Growth / Root or Tuber Development': 7}


In [10]:
df.head()

,crop ID,soil_type,Seedling Stage,MOI,temp,humidity,result
0,4,1,2,1,25,80.0,1
1,4,1,2,2,26,77.0,1
2,4,1,2,3,27,74.0,1
3,4,1,2,4,28,71.0,1
4,4,1,2,5,29,68.0,1


In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

x = df.drop('result', axis=1)
y = df['result']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                448       
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                

In [13]:
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test), callbacks=[earlyStopping])

Epoch 1/100
411/411 [==============================] - 3s 4ms/step - loss: 0.7390 - accuracy: 0.6252 - val_loss: 0.2491 - val_accuracy: 0.7645
Epoch 2/100
411/411 [==============================] - 1s 4ms/step - loss: 0.2485 - accuracy: 0.7725 - val_loss: 0.0407 - val_accuracy: 0.8623
Epoch 3/100
411/411 [==============================] - 1s 3ms/step - loss: 0.0707 - accuracy: 0.8192 - val_loss: -0.1294 - val_accuracy: 0.8730
Epoch 4/100
411/411 [==============================] - 1s 3ms/step - loss: -0.0693 - accuracy: 0.8275 - val_loss: -0.3963 - val_accuracy: 0.8736
Epoch 5/100
411/411 [==============================] - 1s 4ms/step - loss: -0.3010 - accuracy: 0.8287 - val_loss: -1.2679 - val_accuracy: 0.8538
Epoch 6/100
411/411 [==============================] - 1s 4ms/step - loss: -0.7588 - accuracy: 0.8376 - val_loss: -2.1310 - val_accuracy: 0.8660
Epoch 7/100
411/411 [==============================] - 2s 4ms/step - loss: -3.2779 - accuracy: 0.8296 - val_loss: -7.5907 - val_accurac

In [15]:
model.evaluate(x_test, y_test)

103/103 [==============================] - 0s 2ms/step - loss: -220645.6562 - accuracy: 0.8343


[-220645.65625, 0.8342978954315186]

In [16]:
model.save('crop_model.keras')

In [17]:
testing = pd.DataFrame([[4,1,2,5, 29,68]], columns=['crop ID', 'soil_type', 'Seedling Stage', 'fertilizer', 'temperature', 'humidity'])
model.predict(testing)


1/1 [==============================] - 0s 141ms/step


array([[0.51339364]], dtype=float32)